# Assignment 2
## Vdovenko Anton

### Data Pre-Processing

In [32]:
import pandas as pd
import scipy
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, roc_auc_score, precision_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, StackingClassifier


In [33]:
loanStats = pd.read_csv('LoanStats_2016Q4.csv', skiprows = 1)
rejectStats = pd.read_csv('RejectStats_2016Q4.csv', skiprows = 1)

In [34]:
print(loanStats.shape)
print(loanStats.columns)

(103548, 144)
Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'orig_projected_additional_accrued_interest',
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'debt_settlement_flag', 'debt_settlement_flag_date',
       'settlement_status', 'settlement_date', 'settlement_amount',
       'settlement_percentage', 'settlement_term'],
      dtype='object', length=144)


In [35]:
print(rejectStats.shape)
print(rejectStats.columns)

(1404490, 9)
Index(['Amount Requested', 'Application Date', 'Loan Title', 'Risk_Score',
       'Debt-To-Income Ratio', 'Zip Code', 'State', 'Employment Length',
       'Policy Code'],
      dtype='object')


In [36]:
loanStats[-3:]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
103545,NaN,NaN,14000.0,14000.0,14000.0,60 months,14.49%,329.33,C,C4,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
103546,Total amount funded in policy code 1: 1465324575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103547,Total amount funded in policy code 2: 521953170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Drop last two rows

In [37]:
loanStats = loanStats.drop(loanStats.index[-2:])

Drop first two columns since they are empty

In [38]:
loanStats.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'orig_projected_additional_accrued_interest',
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'debt_settlement_flag', 'debt_settlement_flag_date',
       'settlement_status', 'settlement_date', 'settlement_amount',
       'settlement_percentage', 'settlement_term'],
      dtype='object', length=144)

In [39]:
loanStats = loanStats.drop(['id', 'member_id'], axis = 1)

In [40]:
print(loanStats.shape)
print(loanStats.columns)
print(rejectStats.shape)
print(rejectStats.columns)

(103546, 142)
Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       ...
       'orig_projected_additional_accrued_interest',
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'debt_settlement_flag', 'debt_settlement_flag_date',
       'settlement_status', 'settlement_date', 'settlement_amount',
       'settlement_percentage', 'settlement_term'],
      dtype='object', length=142)
(1404490, 9)
Index(['Amount Requested', 'Application Date', 'Loan Title', 'Risk_Score',
       'Debt-To-Income Ratio', 'Zip Code', 'State', 'Employment Length',
       'Policy Code'],
      dtype='object')


In [41]:
print(loanStats['loan_amnt'].describe())
print(rejectStats['Amount Requested'].describe())

count    103546.000000
mean      14151.435835
std        9215.032376
min        1000.000000
25%        7000.000000
50%       12000.000000
75%       20000.000000
max       40000.000000
Name: loan_amnt, dtype: float64
count    1.404490e+06
mean     1.293313e+04
std      1.567272e+04
min      0.000000e+00
25%      4.000000e+03
50%      1.000000e+04
75%      2.000000e+04
max      3.000000e+05
Name: Amount Requested, dtype: float64


In [42]:
print(loanStats['title'].value_counts())
print(rejectStats['Loan Title'].value_counts())

Debt consolidation         59749
Credit card refinancing    20528
Home improvement            7393
Other                       7383
Major purchase              2470
Medical expenses            1468
Business                    1216
Car financing               1162
Vacation                     829
Moving and relocation        772
Home buying                  475
Green loan                    97
Name: title, dtype: int64
debt_consolidation         359782
Debt consolidation         324415
other                      146389
credit_card                 96179
Other                       71882
Credit card refinancing     67381
home_improvement            44136
Home improvement            31054
car                         29480
major_purchase              29301
Car financing               22788
moving                      20836
medical                     20814
Medical expenses            19315
Business Loan               18575
Moving and relocation       17616
Major purchase              17505


There are only 9 columns in reject data, let's try to match loan data with reject data:<br/>
* 'loan_amnt' in loanStats should match with 'Amount Requested' in rejectStats.
* 'issue_d' in loanStats is the issue date, it should be later than 'Application Date' in rejectStats, let's ignore this pair for now.
* 'title' in loanStats should match with 'Loan Title' in rejectStats. However, we need to deal with upper/lower case and space/underscore issues.
* There is no column in loanStats matches with 'Risk_Score' in rejectStats, would be very useful if there was one.
* 'dti' in loanStats matches with 'Debt-To-Income Ratio' in rejectStats.
* 'zip_code' and 'addr_state' in loanStats match with 'Zip Code' and 'State' in rejectStats respectively. Zip codes are only available in first 3 digits, and we might need to get external data for the demographics about the zip code.
* 'emp_length' in loanStats matches with 'Employment Length' in rejectStats.
* 'policy_code' in loanStats matches with 'Policy Code' in rejectStats.

In [43]:
print(loanStats['emp_length'].value_counts())
print(rejectStats['Employment Length'].value_counts())

10+ years    35981
2 years       9652
3 years       8244
< 1 year      7591
1 year        6707
5 years       6292
4 years       6249
6 years       4786
8 years       3951
9 years       3870
7 years       3346
Name: emp_length, dtype: int64
< 1 year     1022412
5 years       274322
10+ years      16782
2 years         5248
3 years         4502
1 year          4389
4 years         3314
6 years         2494
8 years         1953
7 years         1918
9 years         1586
Name: Employment Length, dtype: int64


In [44]:
print(loanStats['policy_code'].value_counts())
print(rejectStats['Policy Code'].value_counts())

1.0    103546
Name: policy_code, dtype: int64
0    1403661
2        829
Name: Policy Code, dtype: int64


However, policy code is useless since it carries different meaning in the two datasets, so we should ignore this pair of columns.<br/>
Let's now build two new dataframes, and keep their columns consistent.

In [45]:
approved = pd.DataFrame()
rejected = pd.DataFrame()

Copy values from 'loan_amnt' and 'Amount Requested' directly to the new dataframes

In [46]:
approved['amount'] = loanStats['loan_amnt']
rejected['amount'] = rejectStats['Amount Requested']

Using lambda functions to convert strings, first make all characters lower cases, and split them by space, then join them using '\_'. While joining, remove words like 'and', 'expenses', 'financing', 'loan', 'refinancing', as these words are not important in reasons for loans. To further consolidate the reasons, we merge 'housing' to 'home\_buying', 'moving' to 'moving\_relocation', 'renewable\_energy' to 'green', and 'small\_business' to 'business'.

In [47]:
stop_words = ['and', 'expenses', 'financing', 'loan', 'refinancing']
approved['reason'] = loanStats['title'].apply(lambda x: 'other' if type(x) != str else '_'.join([i for i in x.lower().split() if i not in stop_words]))
rejected['reason'] = rejectStats['Loan Title'].apply(lambda x: 'other' if type(x) != str else '_'.join([i for i in x.lower().split() if i not in stop_words]))
convert = {'house': 'home_buying', 'moving': 'moving_relocation', 'renewable_energy': 'green', 'small_business': 'business'}
rejected['reason'] = rejected['reason'].apply(lambda x: convert[x] if x in convert else x)

Copy values from 'dti' and 'Debt-To-Income Ratio', however, to identify Not-a-Number floats, we test x == x, in it's a NaN, we set it to the maximum in the dataset.

In [48]:
approved['debt_to_income'] = loanStats['dti'].apply(lambda x: max(0.0, min(x / 100, 1.0)) if x == x else 1.0)
rejected['debt_to_income'] = rejectStats['Debt-To-Income Ratio'].apply(lambda x: max(0.0, min(float(x[:-1]) / 100, 1.0)))

Keep the first 3 digits of the zip codes since the last 2 digits are masked, however, it is unnecessary to convert them to numbers.

In [49]:
approved['zip3'] = loanStats['zip_code'].apply(lambda x: x[:3] if type(x) == str else 'N/A')
rejected['zip3'] = rejectStats['Zip Code'].apply(lambda x: x[:3] if type(x) == str else 'N/A')
approved['state'] = loanStats['addr_state']
rejected['state'] = rejectStats['State']

Convert the employment length to numerical values, if it is not specified, or less than a year, we take them as 0.

In [50]:
approved['employ_length'] = loanStats['emp_length'].apply(lambda x: 0 if type(x) != str or x[:3] == '< 1' else int(x[:2]))
rejected['employ_length'] = rejectStats['Employment Length'].apply(lambda x: 0 if type(x) != str or x[:3] == '< 1' else int(x[:2]))

The column 'reason' is now the only nominal column, we may use function get\_dummies, drop 'other' is recommended since 'reason\_=\_other' is fuzzy, and should be expressed by 0 in all other reason columns.

In [51]:
approved = pd.concat([approved, pd.get_dummies(approved['reason'], prefix = 'reason', prefix_sep = '_=_').drop('reason_=_other', axis = 1)], axis = 1).drop('reason', axis = 1)
rejected = pd.concat([rejected, pd.get_dummies(rejected['reason'], prefix = 'reason', prefix_sep = '_=_').drop('reason_=_other', axis = 1)], axis = 1).drop('reason', axis = 1)

Add the response column to the two dataframes.

In [52]:
approved['approved'] = 1
rejected['approved'] = 0

Convert the two dataframes to numpy arrays.

In [53]:
data_pos = approved.drop(['zip3', 'state'], axis = 1).values
data_neg = rejected.drop(['zip3', 'state'], axis = 1).values

In [54]:
print(data_pos.shape)
print(data_neg.shape)

(103546, 15)
(1404490, 15)


### [Question 1] Build Logistic Regression Models and Naive Bayes Models

Build the two basic classification models

In [55]:
merged = pd.concat([approved,rejected])
merged = merged.drop(["zip3","state"], axis = 1)
merged.head(1)

,amount,debt_to_income,employ_length,reason_=_business,reason_=_car,reason_=_credit_card,reason_=_debt_consolidation,reason_=_green,reason_=_home_buying,reason_=_home_improvement,reason_=_major_purchase,reason_=_medical,reason_=_moving_relocation,reason_=_vacation,approved
0,4500.0,0.2993,6,0,0,1,0,0,0,0,0,0,0,0,1


In [56]:
x = merged.iloc[:,:-1]
y = merged["approved"].values

In [57]:
# Naive Bayes

clf = GaussianNB()
scores = cross_validate(clf, x, y, cv=5,
                        scoring=('accuracy', 'f1', "roc_auc","precision","recall"))
                        
for i in scores:
    print(i,":",scores[i].mean().round(2))
b = 0.5
f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores["test_recall"].mean()))
print("f_beta:",f_beta.round(2),"beta:",b)

test_recall : 0.45
score_time : 0.53
test_precision : 0.68
test_f1 : 0.54
test_accuracy : 0.95
fit_time : 0.45
test_roc_auc : 0.84
f_beta: 0.62 beta: 0.5


In [58]:
# Chosse Threshold to max Fb
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

clf = GaussianNB()
clf.fit(X_train, y_train)
THRESHOLD = 0.4
preds = np.where(clf.predict_proba(X_test)[:,1] > THRESHOLD, 1, 0)

z = pd.DataFrame(data=[accuracy_score(y_test, preds), recall_score(y_test, preds),
                   precision_score(y_test, preds), roc_auc_score(y_test, preds)], 
             index=["accuracy", "recall", "precision", "roc_auc_score"])
b = 0.5
f_beta = (1 + b**2) * ((precision_score(y_test, preds)*recall_score(y_test, preds))/((b**2 * precision_score(y_test, preds)) + recall_score(y_test, preds)))
print("f_beta:",f_beta.round(2),"beta:",b)
print(z)

f_beta: 0.63 beta: 0.5
                      0
accuracy       0.949097
recall         0.479468
precision      0.680983
roc_auc_score  0.731500


In [59]:
# distributions = dict(C = scipy.stats.uniform(loc=0, scale=4),penalty=['l2'])
# logistic = LogisticRegression(random_state = 0)
# clf = RandomizedSearchCV(logistic, distributions, random_state=0)
# search = clf.fit(x, y)
# search.best_params_
# C = 3.567092003128319

In [60]:
# Logistic Regression Classifier
x_scaled = MinMaxScaler().fit_transform(x)

clf = LogisticRegression(C = 3.567092003128319, random_state=0)
scores = cross_validate(clf, x_scaled, y, cv=5,
                        scoring=('accuracy', 'f1', "roc_auc","precision","recall"))
                        
for i in scores:
    print(i,":",scores[i].mean().round(2))
b = 0.5
f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores["test_recall"].mean()))
print("f_beta:",f_beta.round(2),"beta:",b)

test_recall : 0.33
score_time : 0.36
test_precision : 0.7
test_f1 : 0.45
test_accuracy : 0.94
fit_time : 5.45
test_roc_auc : 0.85
f_beta: 0.58 beta: 0.5


In [61]:
# Chosse Threshold to max Fb

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

clf = LogisticRegression(class_weight="balanced",C = 3.567092003128319, random_state=0)
clf.fit(X_train, y_train)
THRESHOLD = 0.75
preds = np.where(clf.predict_proba(X_test)[:,1] > THRESHOLD, 1, 0)

z = pd.DataFrame(data=[accuracy_score(y_test, preds), recall_score(y_test, preds),
                   precision_score(y_test, preds), roc_auc_score(y_test, preds)], 
             index=["accuracy", "recall", "precision", "roc_auc_score"])
b = 0.5
f_beta = (1 + b**2) * ((precision_score(y_test, preds)*recall_score(y_test, preds))/((b**2 * precision_score(y_test, preds)) + recall_score(y_test, preds)))
print("f_beta:",f_beta.round(2),"beta:",b)
print(z)

f_beta: 0.62 beta: 0.5
                      0
accuracy       0.948259
recall         0.490617
precision      0.664211
roc_auc_score  0.736215


|  | Logistic Regression with C = 3.6   |Naive Bayes  |
|------|------|-----|
|   Precision | 0.7|0.68|
|   Recall | 0.33|0.45|
|   Chosen Beta | 0.5|
|   Fb | 0.58|0.62|
|   AUC under ROC | 0.85|0.84|

|  | Logistic Regression with Treshold 0.75|Naive Bayes with Treshold 0.4  |
|------|------|-----|
|   Precision | 0.66|0.68|
|   Recall | 0.49|0.48|
|   Chosen Beta | 0.5|
|   Fb | 0.62|0.63|
|   AUC under ROC | 0.74|0.73|

In [62]:
# Precision = TP/(TP+FP), Recall = TP/(TP + FN). False Positive is the case we give money to someone you shouldn't
# and most likely face huge loss, False Negative is the case we didnt give money to someone we should have and miss profit.
# Both are important but having as less loss as possible is crucial so we prioritize having less False Positives
# and hence high Precision. For F Beta score standard practice when we put more importance to Precision is to choose value
# of beta = 0.5. When we look at our final table we can see that AUC is close to equal in both models, but Naive Bayes
# has higher Fb score, hence we choose Naive Bayes.

### [Question 2] Build Classification Models over GMM

Build GMM models + classification models

In [63]:
# GaussianMixture 5 classes for NB

gmm = GaussianMixture(n_components=5, random_state=0).fit_predict(x)
clusters = pd.DataFrame(gmm,columns=["cluster"])
X = pd.merge(x,clusters, on=[x.index,clusters.index])
X = X.drop(["key_0","key_1"], axis=1)

# Naive Bayes on 5 Clusters

clf = GaussianNB()
scores = cross_validate(clf, X, y, cv=5,
                        scoring=('accuracy', 'f1', "roc_auc","precision","recall"))
print("Naive Bayes:")                        
for i in scores:
    print(i,":",scores[i].mean().round(2))
b = 0.5
f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores["test_recall"].mean()))
print("f_beta:",f_beta.round(2),"beta:",b)

Naive Bayes:
test_recall : 0.47
score_time : 0.55
test_precision : 0.69
test_f1 : 0.55
test_accuracy : 0.95
fit_time : 0.47
test_roc_auc : 0.84
f_beta: 0.63 beta: 0.5


In [64]:
# GaussianMixture 10 classes for NB

gmm = GaussianMixture(n_components=10, random_state=0).fit_predict(x)
clusters = pd.DataFrame(gmm,columns=["cluster"])
X = pd.merge(x,clusters, on=[x.index,clusters.index])
X = X.drop(["key_0","key_1"], axis=1)

# Naive Bayes on 5 Clusters

clf = GaussianNB()
scores = cross_validate(clf, X, y, cv=5,
                        scoring=('accuracy', 'f1', "roc_auc","precision","recall"))
print("Naive Bayes:")                        
for i in scores:
    print(i,":",scores[i].mean().round(2))
b = 0.5
f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores["test_recall"].mean()))
print("f_beta:",f_beta.round(2),"beta:",b)

Naive Bayes:
test_recall : 0.46
score_time : 0.53
test_precision : 0.69
test_f1 : 0.55
test_accuracy : 0.95
fit_time : 0.47
test_roc_auc : 0.84
f_beta: 0.63 beta: 0.5


In [65]:
# GaussianMixture 20 classes for NB

gmm = GaussianMixture(n_components=20, random_state=0).fit_predict(x)
clusters = pd.DataFrame(gmm,columns=["cluster"])
X = pd.merge(x,clusters, on=[x.index,clusters.index])
X = X.drop(["key_0","key_1"], axis=1)

# Naive Bayes on 5 Clusters

clf = GaussianNB()
scores = cross_validate(clf, X, y, cv=5,
                        scoring=('accuracy', 'f1', "roc_auc","precision","recall"))
print("Naive Bayes:")                        
for i in scores:
    print(i,":",scores[i].mean().round(2))
b = 0.5
f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores["test_recall"].mean()))
print("f_beta:",f_beta.round(2),"beta:",b)

Naive Bayes:
test_recall : 0.46
score_time : 0.54
test_precision : 0.69
test_f1 : 0.55
test_accuracy : 0.95
fit_time : 0.46
test_roc_auc : 0.84
f_beta: 0.63 beta: 0.5


In [66]:
# GaussianMixture 5 classes for LR
x_scaled = StandardScaler().fit_transform(x)
gmm = GaussianMixture(n_components=5, random_state=0).fit_predict(x_scaled)
clusters = pd.DataFrame(gmm,columns=["cluster"])
X = pd.merge(x,clusters, on=[x.index,clusters.index])
X = X.drop(["key_0","key_1"], axis=1)

# Logistic Regression Classifier on 5 Clusters
x_scaled = MinMaxScaler().fit_transform(X)

clf = LogisticRegression(C = 3.567092003128319, random_state=0)
scores = cross_validate(clf, x_scaled, y, cv=5,
                        scoring=('accuracy', 'f1', "roc_auc","precision","recall"))

print("Logistic Regression:")                        
for i in scores:
    print(i,":",scores[i].mean().round(2))
b = 0.5
f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores["test_recall"].mean()))
print("f_beta:",f_beta.round(2),"beta:",b)

Logistic Regression:
test_recall : 0.33
score_time : 0.35
test_precision : 0.7
test_f1 : 0.45
test_accuracy : 0.94
fit_time : 5.58
test_roc_auc : 0.85
f_beta: 0.58 beta: 0.5


In [67]:
# GaussianMixture 10 classes for LR
x_scaled = StandardScaler().fit_transform(x)
gmm = GaussianMixture(n_components=10, random_state=0).fit_predict(x_scaled)
clusters = pd.DataFrame(gmm,columns=["cluster"])
X = pd.merge(x,clusters, on=[x.index,clusters.index])
X = X.drop(["key_0","key_1"], axis=1)

# Logistic Regression Classifier on 5 Clusters
x_scaled = MinMaxScaler().fit_transform(X)

clf = LogisticRegression(C = 3.567092003128319, random_state=0)
scores = cross_validate(clf, x_scaled, y, cv=5,
                        scoring=('accuracy', 'f1', "roc_auc","precision","recall"))

print("Logistic Regression:")                        
for i in scores:
    print(i,":",scores[i].mean().round(2))
b = 0.5
f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores["test_recall"].mean()))
print("f_beta:",f_beta.round(2),"beta:",b)

Logistic Regression:
test_recall : 0.29
score_time : 0.36
test_precision : 0.7
test_f1 : 0.41
test_accuracy : 0.94
fit_time : 5.69
test_roc_auc : 0.87
f_beta: 0.55 beta: 0.5


In [68]:
# GaussianMixture 20 classes for LR
x_scaled = StandardScaler().fit_transform(x)
gmm = GaussianMixture(n_components=20, random_state=0).fit_predict(x_scaled)
clusters = pd.DataFrame(gmm,columns=["cluster"])
X = pd.merge(x,clusters, on=[x.index,clusters.index])
X = X.drop(["key_0","key_1"], axis=1)

# Logistic Regression Classifier on 5 Clusters
x_scaled = MinMaxScaler().fit_transform(X)

clf = LogisticRegression(C = 3.567092003128319, random_state=0)
scores = cross_validate(clf, x_scaled, y, cv=5,
                        scoring=('accuracy', 'f1', "roc_auc","precision","recall"))

print("Logistic Regression:")                        
for i in scores:
    print(i,":",scores[i].mean().round(2))
b = 0.5
f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores["test_recall"].mean()))
print("f_beta:",f_beta.round(2),"beta:",b)

Logistic Regression:
test_recall : 0.33
score_time : 0.34
test_precision : 0.7
test_f1 : 0.45
test_accuracy : 0.94
fit_time : 6.19
test_roc_auc : 0.85
f_beta: 0.58 beta: 0.5


|  |    |Logistic Regression With C=3.6  |Naive Bayes  |
|------|------|-----|-----|
|    | Precision|0.7|0.69|
|    | Recall|0.33|0.47|
| 5 clusters   | Fb b=0.5|0.58|0.63|
|    | AUC under ROC|0.85|0.84|

|  |    |Logistic Regression With C=3.6  |Naive Bayes  |
|------|------|-----|-----|
|    | Precision|0.7|0.69|
|    | Recall|0.29|0.46|
| 10 clusters   | Fb b=0.5|0.55|0.63|
|    | AUC under ROC|0.87|0.84|


|  |    |Logistic Regression With C=3.6  |Naive Bayes  |
|------|------|-----|-----|
|    | Precision|0.7|0.69|
|    | Recall|0.33|0.46|
| 20 clusters   | Fb b=0.5|0.58|0.63|
|    | AUC under ROC|0.85|0.84|



In [214]:
# Naive Bayes is very stable the only difference we can see is that Recall droped on 0.01 when number of clusters increased.
# Logistic Regression has best AUC with 10 clusters, but best Fb with 5 and 20 clusters. Overall if we compare with Q1
# the Naive Bayes model Fb increased by 0.01, a minor performance improvement.

### [Question 3] Build Ensemble Models

### Bagging Models

In [69]:
# Baggig Naive Bayes Classifier

NB_clf = GaussianNB()
nb_bagging_model = BaggingClassifier(base_estimator=NB_clf, n_estimators=100, random_state=12)


def bagging_ensemble(model):    
    scores = cross_validate(model, x, y, cv=5,scoring=('accuracy', 'f1', "roc_auc","precision","recall"))
    print(model)
                          
    for i in scores:
        print(i,":",scores[i].mean().round(2))
    
    b = 0.5
    f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores        ["test_recall"].mean()))
     
    print("f_beta:",f_beta.round(2),"beta:",b)
    


bagging_ensemble(nb_bagging_model)


BaggingClassifier(base_estimator=GaussianNB(priors=None, var_smoothing=1e-09),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=100, n_jobs=None,
                  oob_score=False, random_state=12, verbose=0,
                  warm_start=False)
test_recall : 0.45
score_time : 21.55
test_precision : 0.68
test_f1 : 0.54
test_accuracy : 0.95
fit_time : 53.33
test_roc_auc : 0.84
f_beta: 0.62 beta: 0.5


In [209]:
# Bagging Logistic Regression Classifier
x_scaled = MinMaxScaler().fit_transform(x)
LogReg_clf = LogisticRegression(C = 3.567092003128319)

logreg_bagging_model = BaggingClassifier(base_estimator=LogReg_clf, n_estimators=5, random_state=12)


def bagging_ensemble(model):
    
    scores = cross_validate(model, x_scaled, y, cv=5,scoring=('accuracy', 'f1', "roc_auc","precision","recall"))
    print(model)
                          
    for i in scores:
        print(i,":",scores[i].mean().round(2))
    
    b = 0.5
    f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores        ["test_recall"].mean()))
     
    print("f_beta:",f_beta.round(2),"beta:",b)
    

bagging_ensemble(logreg_bagging_model)


BaggingClassifier(base_estimator=LogisticRegression(C=3.567092003128319),
                  n_estimators=5, random_state=12)
fit_time : 45.22
score_time : 1.2
test_accuracy : 0.94
test_f1 : 0.45
test_roc_auc : 0.85
test_precision : 0.7
test_recall : 0.33
f_beta: 0.57 beta: 0.5


In [70]:
# Bagging Decesion Tree Classifier


DTree_clf = DecisionTreeClassifier()

dtree_bagging_model = BaggingClassifier(base_estimator=DTree_clf, n_estimators=50, random_state=12)


def bagging_ensemble(model):
    
    scores = cross_validate(model, x, y, cv=5,scoring=('accuracy', 'f1', "roc_auc","precision","recall"))
    print(model)
                          
    for i in scores:
        print(i,":",scores[i].mean().round(2))
    
    b = 0.5
    f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores        ["test_recall"].mean()))
     
    print("f_beta:",f_beta.round(2),"beta:",b)
    


bagging_ensemble(dtree_bagging_model)


BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=None,


In [127]:
# Extra Trees Classifier

extra_trees = ExtraTreesClassifier(n_estimators=100, random_state=12)

def bagging_ensemble(model):
    
    scores = cross_validate(model, x, y, cv=5,scoring=('accuracy', 'f1', "roc_auc","precision","recall"))
    print(model)
                          
    for i in scores:
        print(i,":",scores[i].mean().round(2))
    
    b = 0.5
    f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores        ["test_recall"].mean()))
     
    print("f_beta:",f_beta.round(2),"beta:",b)
    

bagging_ensemble(extra_trees)

ExtraTreesClassifier(random_state=12)
fit_time : 105.38
score_time : 17.89
test_accuracy : 0.95
test_f1 : 0.64
test_roc_auc : 0.9
test_precision : 0.67
test_recall : 0.61
f_beta: 0.65 beta: 0.5


### Boosting model

In [71]:
# AdaBoost model
num_estimators = [60,80]

for i in num_estimators:
    ada_boost = AdaBoostClassifier(n_estimators=i, random_state=12)
    scores = cross_validate(ada_boost, x, y, cv=5,scoring=('accuracy', 'f1', "roc_auc","precision","recall"))
    print("Results for {} estimators:".format(i))
    for i in scores:
        print(i,":",scores[i].mean().round(2))
    
    b = 0.5
    f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores        ["test_recall"].mean()))
     
    print("f_beta:",f_beta.round(2),"beta:",b)

Results for 60 estimators:
test_recall : 0.74
score_time : 5.34
test_precision : 0.71
test_f1 : 0.73
test_accuracy : 0.96
fit_time : 48.52
test_roc_auc : 0.96
f_beta: 0.72 beta: 0.5
Results for 80 estimators:
test_recall : 0.73
score_time : 6.78
test_precision : 0.72
test_f1 : 0.72
test_accuracy : 0.96
fit_time : 62.58
test_roc_auc : 0.96
f_beta: 0.72 beta: 0.5


Stacking Model

In [212]:
# Voting Model
voting_clf = VotingClassifier(estimators=[('NB', NB_clf), ('DTree', DTree_clf), ('LogReg', LogReg_clf)], voting='soft')
scores = cross_validate(voting_clf, x, y, cv=5,scoring=('accuracy', 'f1', "roc_auc","precision","recall"))

for i in scores:
    print(i,":",scores[i].mean().round(2))
    
b = 0.5
f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores        ["test_recall"].mean()))
     
print("f_beta:",f_beta.round(2),"beta:",b)

fit_time : 24.56
score_time : 1.72
test_accuracy : 0.95
test_f1 : 0.53
test_roc_auc : 0.9
test_precision : 0.71
test_recall : 0.42
f_beta: 0.62 beta: 0.5


In [74]:
# Stacking Model
LogReg_clf = LogisticRegression(C = 3.567092003128319)
NB_clf = GaussianNB()
DTree_clf = DecisionTreeClassifier()
estimators = [('NB', NB_clf), ('DTree', DTree_clf), ("LogReg",LogReg_clf)]

stacking_clf = StackingClassifier(estimators=estimators, final_estimator=RandomForestClassifier())
scores = cross_validate(stacking_clf, x, y, cv=2,scoring=('accuracy', 'f1', "roc_auc","precision","recall"))

for i in scores:
    print(i,":",scores[i].mean().round(2))
    
b = 0.5
f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores        ["test_recall"].mean()))
     
print("f_beta:",f_beta.round(2),"beta:",b)

test_recall : 0.63
score_time : 37.87
test_precision : 0.69
test_f1 : 0.66
test_accuracy : 0.96
fit_time : 227.75
test_roc_auc : 0.91
f_beta: 0.68 beta: 0.5


|  | Bagging   |Boosting  |Stacking  |
|------|------|-----|-----|
|  Base Classifier  | DecesionTree|AdaBoost|DecesionTree,LogReg,NaiveBayes,RandomForest|
|  Precision  | 0.68|0.72|0.69|
| Recall   | 0.64|0.73|0.63|
|   Fb b=0.5 | 0.67|0.72|0.68|
|   AUC under ROC | 0.92|0.96|0.91|



In [ ]:
# Boosting model by far outpreformed others, Its the best model we have for now, also it was faster to compute than others
# models in Q3, If increase number of estimators in Stacking or Bagging it might increase performance as well,
# but it requires more powerfull hardware than in my disposal. Boosting algorithms was a king in a Kaggle for a reason.

### [Question 4] Build Ensemble Models with External Data

getting external data<br/>
https://www.irs.gov/pub/irs-soi/16zpallagi.csv<br/>
more details:<br/>
https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi

In [76]:
f = open('16zpallagi.csv', 'r')
f.readline()
count = {}
for l in f.readlines():
    s = l.split(',')
    if s[2] == '0':
        count[s[1], int(s[3])] = int(s[4])
    else:
        count['%05d' % int(s[2]), int(s[3])] = int(s[4])
agg_count = {}
for k, v in count.items():
    if k[0][:3] not in agg_count:
        agg_count[k[0][:3]] = [0] * 7
    agg_count[k[0][:3]][k[1]] += v
f.close()

In [77]:
# Assume incomes bracketed are [12500, 37500, 62500, 87500, 150000, 400000]
# 1 = $1 under $25,000
# 2 = $25,000 under $50,000
# 3 = $50,000 under $75,000
# 4 = $75,000 under $100,000
# 5 = $100,000 under $200,000
# 6 = $200,000 or more

bracket_income = [0, 12500, 37500, 62500, 87500, 150000, 400000]
estimated_zip_income = {}
for k, v in agg_count.items():
    sumn, sumd = 0, 0
    for i in range(1, 7):
        sumn += bracket_income[i] * v[i]
        sumd += v[i]
    estimated_zip_income[k] = sumn / sumd

In [78]:
approved.insert(4, 'estimated_income', approved[['zip3', 'state']].apply(lambda x: estimated_zip_income[x[0]] if x[0] in estimated_zip_income else estimated_zip_income[x[1]] if x[1] in estimated_zip_income else 0, axis = 1))
rejected.insert(4, 'estimated_income', rejected[['zip3', 'state']].apply(lambda x: estimated_zip_income[x[0]] if x[0] in estimated_zip_income else estimated_zip_income[x[1]] if x[1] in estimated_zip_income else 0, axis = 1))
data_pos = approved.drop(['zip3', 'state'], axis = 1).values
data_neg = rejected.drop(['zip3', 'state'], axis = 1).values

In [79]:
merged = pd.concat([approved,rejected])
merged = merged.drop(["zip3","state"], axis = 1)
merged.head(1)
x = merged.iloc[:,:-1]
y = merged["approved"].values

### Boosting Model

In [152]:
# AdaBoost model
num_estimators = [20, 40]

for i in num_estimators:
    ada_boost = AdaBoostClassifier(n_estimators=i, random_state=12)
    scores = cross_validate(ada_boost, x, y, cv=5,scoring=('accuracy', 'f1', "roc_auc","precision","recall"))
    print("Results for {} estimators:".format(i))
    for i in scores:
        print(i,":",scores[i].mean().round(2))
    
    b = 0.5
    f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores        ["test_recall"].mean()))
     
    print("f_beta:",f_beta.round(2),"beta:",b)

Results for 20 estimators:
fit_time : 32.92
score_time : 3.75
test_accuracy : 0.96
test_f1 : 0.71
test_roc_auc : 0.95
test_precision : 0.71
test_recall : 0.71
f_beta: 0.71 beta: 0.5
Results for 40 estimators:
fit_time : 57.73
score_time : 5.71
test_accuracy : 0.96
test_f1 : 0.72
test_roc_auc : 0.96
test_precision : 0.71
test_recall : 0.73
f_beta: 0.71 beta: 0.5


### Stacking Model

In [153]:
# Voting Model
voting_clf = VotingClassifier(estimators=[('NB', NB_clf), ('DTree', DTree_clf), ('LogReg', LogReg_clf)], voting='soft')
scores = cross_validate(voting_clf, x, y, cv=5,scoring=('accuracy', 'f1', "roc_auc","precision","recall"))

for i in scores:
    print(i,":",scores[i].mean().round(2))
    
b = 0.5
f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores        ["test_recall"].mean()))
     
print("f_beta:",f_beta.round(2),"beta:",b)

fit_time : 15.46
score_time : 1.36
test_accuracy : 0.95
test_f1 : 0.47
test_roc_auc : 0.87
test_precision : 0.75
test_recall : 0.35
f_beta: 0.61 beta: 0.5


In [80]:
# Stacking Model
LogReg_clf = LogisticRegression(C = 3.567092003128319)
NB_clf = GaussianNB()
DTree_clf = DecisionTreeClassifier()
estimators = [('NB', NB_clf), ('DTree', DTree_clf), ("LogReg",LogReg_clf)]

stacking_clf = StackingClassifier(estimators=estimators, final_estimator=RandomForestClassifier())
scores = cross_validate(stacking_clf, x, y, cv=2,scoring=('accuracy', 'f1', "roc_auc","precision","recall"))

for i in scores:
    print(i,":",scores[i].mean().round(2))
    
b = 0.5
f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores        ["test_recall"].mean()))
     
print("f_beta:",f_beta.round(2),"beta:",b)

test_recall : 0.56
score_time : 41.63
test_precision : 0.67
test_f1 : 0.61
test_accuracy : 0.95
fit_time : 234.7
test_roc_auc : 0.89
f_beta: 0.65 beta: 0.5


### Bagging Model

In [155]:
# Bagging Decesion Tree Classifier


DTree_clf = DecisionTreeClassifier()

dtree_bagging_model = BaggingClassifier(base_estimator=DTree_clf, n_estimators=20, random_state=12)


def bagging_ensemble(model):
    
    scores = cross_validate(model, x, y, cv=5,scoring=('accuracy', 'f1', "roc_auc","precision","recall"))
    print(model)
                          
    for i in scores:
        print(i,":",scores[i].mean().round(2))
    
    b = 0.5
    f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores        ["test_recall"].mean()))
     
    print("f_beta:",f_beta.round(2),"beta:",b)
    


bagging_ensemble(dtree_bagging_model)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=20,
                  random_state=12)
fit_time : 173.86
score_time : 5.25
test_accuracy : 0.96
test_f1 : 0.68
test_roc_auc : 0.93
test_precision : 0.73
test_recall : 0.64
f_beta: 0.71 beta: 0.5


In [156]:
# Baggig Naive Bayes Classifier


NB_clf = GaussianNB()
nb_bagging_model = BaggingClassifier(base_estimator=NB_clf, n_estimators=50, random_state=12)


def bagging_ensemble(model):    
    scores = cross_validate(model, x, y, cv=5,scoring=('accuracy', 'f1', "roc_auc","precision","recall"))
    print(model)
                          
    for i in scores:
        print(i,":",scores[i].mean().round(2))
    
    b = 0.5
    f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores        ["test_recall"].mean()))
     
    print("f_beta:",f_beta.round(2),"beta:",b)
    


bagging_ensemble(nb_bagging_model)

BaggingClassifier(base_estimator=GaussianNB(), n_estimators=50, random_state=12)
fit_time : 49.69
score_time : 21.14
test_accuracy : 0.95
test_f1 : 0.54
test_roc_auc : 0.84
test_precision : 0.67
test_recall : 0.46
f_beta: 0.61 beta: 0.5


In [158]:
# Bagging Logistic Regression Classifier
x_scaled = MinMaxScaler().fit_transform(x)
LogReg_clf = LogisticRegression(C = 3.567092003128319)

logreg_bagging_model = BaggingClassifier(base_estimator=LogReg_clf, n_estimators=5, random_state=12)


def bagging_ensemble(model):
    
    scores = cross_validate(model, x_scaled, y, cv=5,scoring=('accuracy', 'f1', "roc_auc","precision","recall"))
    print(model)
                          
    for i in scores:
        print(i,":",scores[i].mean().round(2))
    
    b = 0.5
    f_beta = (1 + b**2) * ((scores["test_precision"].mean()*scores["test_recall"].mean())/((b**2 * scores["test_precision"].mean()) + scores        ["test_recall"].mean()))
     
    print("f_beta:",f_beta.round(2),"beta:",b)
    

bagging_ensemble(logreg_bagging_model)

BaggingClassifier(base_estimator=LogisticRegression(C=3.567092003128319),
                  n_estimators=5, random_state=12)
fit_time : 45.31
score_time : 1.23
test_accuracy : 0.94
test_f1 : 0.45
test_roc_auc : 0.86
test_precision : 0.7
test_recall : 0.33
f_beta: 0.58 beta: 0.5


|  | Bagging   |Boosting  |Stacking  |
|------|------|-----|-----|
|  Base Classifier  | DecesionTree|AdaBoost|DecesionTree,LogReg,NaiveBayes,RandomForest|
|  Precision  | 0.73|0.71|0.67|
| Recall   | 0.64|0.73|0.56|
|   Fb b=0.5 | 0.71|0.71|0.65|
|   AUC under ROC | 0.93|0.96|0.89|




In [ ]:
# Stacking model became worse, Boosting aprox. same, Bagging has decent leap in performance but still Boosting model is
# preferable even though Fb are same, Boosting model has higher AUC.